Perfect! Let's implement **Step 1.1: Configuration & Logger Setup** with comprehensive prompt engineering notes.[1][2][3][4]

***

## **STEP 1.1: Configuration & Logger Setup**

### **File 1: `backend/app/config.py`**

```python
"""
Configuration Management using Pydantic Settings v2

This module centralizes all application settings using Pydantic's BaseSettings,
which provides automatic environment variable loading, type validation, and 
structured configuration management.

Key Features:
- Automatic .env file loading
- Type validation for all settings
- Support for multiple LLM providers (Groq, Google Gemini)
- Database connection management (PostgreSQL, Redis, Vector DB)
- Security settings (API keys, JWT secrets)
"""

import os
from functools import lru_cache
from typing import Literal, Optional

from pydantic import Field, PostgresDsn, RedisDsn, field_validator
from pydantic_settings import BaseSettings, SettingsConfigDict


class Settings(BaseSettings):
    """
    Application settings loaded from environment variables.
    
    Pydantic automatically:
    1. Reads from .env file
    2. Converts environment variables to correct types
    3. Validates required fields
    4. Provides default values where specified
    """
    
    # =========================================================================
    # APPLICATION SETTINGS
    # =========================================================================
    APP_NAME: str = Field(
        default="Autonomous Multi-Agent System",
        description="Application name for logging and monitoring"
    )
    ENVIRONMENT: Literal["development", "staging", "production"] = Field(
        default="development",
        description="Current environment"
    )
    DEBUG: bool = Field(
        default=True,
        description="Enable debug mode (verbose logging, auto-reload)"
    )
    API_HOST: str = Field(default="0.0.0.0", description="API server host")
    API_PORT: int = Field(default=8000, description="API server port")
    
    # =========================================================================
    # LLM PROVIDER SETTINGS
    # =========================================================================
    # We support multiple LLM providers for flexibility and cost optimization
    # Groq: Fast inference, free tier available
    # Google Gemini: Multimodal capabilities, generous free tier
    
    GROQ_API_KEY: str = Field(
        ...,  # Required field
        description="Groq API key for LLM inference"
    )
    GOOGLE_API_KEY: str = Field(
        ...,
        description="Google AI Studio API key for Gemini models"
    )
    
    DEFAULT_LLM_PROVIDER: Literal["groq", "google"] = Field(
        default="groq",
        description="Default LLM provider to use"
    )
    GROQ_MODEL_NAME: str = Field(
        default="llama-3.3-70b-versatile",
        description="Groq model for agent reasoning"
    )
    GOOGLE_MODEL_NAME: str = Field(
        default="gemini-2.0-flash-exp",
        description="Google model for complex tasks"
    )
    
    # =========================================================================
    # DATABASE SETTINGS (Cloud PostgreSQL)
    # =========================================================================
    # Using cloud PostgreSQL (Neon, Supabase, etc.) for:
    # - Conversation history storage
    # - LangGraph checkpointing (agent state persistence)
    # - User management
    
    POSTGRES_HOST: str = Field(..., description="PostgreSQL host")
    POSTGRES_PORT: int = Field(default=5432, description="PostgreSQL port")
    POSTGRES_DB: str = Field(..., description="Database name")
    POSTGRES_USER: str = Field(..., description="Database user")
    POSTGRES_PASSWORD: str = Field(..., description="Database password")
    POSTGRES_SSL_MODE: str = Field(default="require", description="SSL mode")
    
    # Computed database URL for SQLAlchemy
    @property
    def DATABASE_URL(self) -> str:
        """
        Constructs async PostgreSQL connection string.
        Uses asyncpg driver for async operations with SQLAlchemy.
        """
        return (
            f"postgresql+asyncpg://{self.POSTGRES_USER}:{self.POSTGRES_PASSWORD}"
            f"@{self.POSTGRES_HOST}:{self.POSTGRES_PORT}/{self.POSTGRES_DB}"
            f"?ssl={self.POSTGRES_SSL_MODE}"
        )
    
    # =========================================================================
    # REDIS SETTINGS (Cloud Redis for caching)
    # =========================================================================
    # Redis is used for:
    # - Agent execution state caching
    # - Session management
    # - Rate limiting
    # - Hot reload mechanism (cache frequently accessed DB data)
    
    REDIS_HOST: str = Field(..., description="Redis host")
    REDIS_PORT: int = Field(default=6379, description="Redis port")
    REDIS_PASSWORD: Optional[str] = Field(default=None, description="Redis password")
    REDIS_DB: int = Field(default=0, description="Redis database number")
    REDIS_SSL: bool = Field(default=False, description="Use SSL for Redis")
    
    @property
    def REDIS_URL(self) -> str:
        """
        Constructs Redis connection string.
        Supports both redis:// and rediss:// (SSL) protocols.
        """
        protocol = "rediss" if self.REDIS_SSL else "redis"
        auth = f"default:{self.REDIS_PASSWORD}@" if self.REDIS_PASSWORD else ""
        return f"{protocol}://{auth}{self.REDIS_HOST}:{self.REDIS_PORT}/{self.REDIS_DB}"
    
    ENABLE_CACHE: bool = Field(default=True, description="Enable Redis caching")
    CACHE_TTL: int = Field(default=3600, description="Cache TTL in seconds (1 hour)")
    CACHE_HOT_RELOAD_INTERVAL: int = Field(
        default=300,
        description="Interval to reload cache from PostgreSQL (5 minutes)"
    )
    
    # =========================================================================
    # VECTOR DATABASE SETTINGS
    # =========================================================================
    # Vector DB is used for:
    # - Semantic search in agent memory
    # - Document embeddings
    # - Conversation context retrieval
    
    VECTOR_DB_TYPE: Literal["pinecone", "chromadb"] = Field(
        default="pinecone",
        description="Vector database type"
    )
    
    # Pinecone settings (cloud vector DB with free tier)
    PINECONE_API_KEY: Optional[str] = Field(default=None)
    PINECONE_ENVIRONMENT: Optional[str] = Field(default="us-east-1")
    PINECONE_INDEX_NAME: Optional[str] = Field(default="agent-memory")
    
    # ChromaDB settings (can be self-hosted or cloud)
    CHROMA_HOST: Optional[str] = Field(default="localhost")
    CHROMA_PORT: Optional[int] = Field(default=8000)
    CHROMA_COLLECTION_NAME: Optional[str] = Field(default="agent_embeddings")
    
    # =========================================================================
    # SECURITY SETTINGS
    # =========================================================================
    API_KEY: str = Field(
        ...,
        min_length=32,
        description="API key for authentication"
    )
    SECRET_KEY: str = Field(
        ...,
        min_length=32,
        description="JWT secret key"
    )
    ALGORITHM: str = Field(default="HS256", description="JWT algorithm")
    ACCESS_TOKEN_EXPIRE_MINUTES: int = Field(
        default=30,
        description="JWT token expiration time"
    )
    
    # CORS settings
    CORS_ORIGINS: str = Field(
        default="http://localhost:3000,http://127.0.0.1:3000",
        description="Comma-separated list of allowed CORS origins"
    )
    
    @property
    def CORS_ORIGINS_LIST(self) -> list[str]:
        """Convert comma-separated CORS origins to list."""
        return [origin.strip() for origin in self.CORS_ORIGINS.split(",")]
    
    # =========================================================================
    # AGENT CONFIGURATION
    # =========================================================================
    # These settings control agent behavior and LLM interaction
    
    MAX_AGENT_ITERATIONS: int = Field(
        default=10,
        description="Maximum iterations for agent reasoning loop"
    )
    AGENT_TIMEOUT_SECONDS: int = Field(
        default=120,
        description="Timeout for agent execution (2 minutes)"
    )
    ENABLE_AGENT_MEMORY: bool = Field(
        default=True,
        description="Enable long-term memory for agents"
    )
    SUPERVISOR_MODEL: str = Field(
        default="llama-3.3-70b-versatile",
        description="LLM model for supervisor agent"
    )
    
    # =========================================================================
    # LOGGING SETTINGS
    # =========================================================================
    LOG_LEVEL: Literal["DEBUG", "INFO", "WARNING", "ERROR", "CRITICAL"] = Field(
        default="INFO",
        description="Logging level"
    )
    LOG_FORMAT: Literal["json", "console"] = Field(
        default="json",
        description="Log output format (json for production, console for dev)"
    )
    ENABLE_STRUCTURED_LOGGING: bool = Field(
        default=True,
        description="Enable structlog for structured logging"
    )
    
    # =========================================================================
    # PYDANTIC SETTINGS CONFIGURATION
    # =========================================================================
    model_config = SettingsConfigDict(
        env_file=".env",  # Load from .env file
        env_file_encoding="utf-8",
        case_sensitive=True,  # Environment variables are case-sensitive
        extra="ignore",  # Ignore extra environment variables
    )


@lru_cache()
def get_settings() -> Settings:
    """
    Create and cache settings instance.
    
    Using @lru_cache ensures settings are loaded only once and reused
    across the application, improving performance.
    
    Returns:
        Settings: Cached settings instance
    """
    return Settings()


# Export singleton instance
settings = get_settings()
```

***

### **File 2: `backend/app/utils/__init__.py`**

```python
"""Utility modules for the application."""
```

***

### **File 3: `backend/app/utils/logger.py`**

```python
"""
Structured Logging Setup using structlog

This module configures structlog for production-grade logging with:
- JSON output for production (easily parsed by log aggregators)
- Human-readable console output for development
- Request ID tracking across async operations
- Automatic exception formatting
- Context-aware logging (attaches metadata to all logs in request scope)

Why structlog over standard logging?
1. Structured data: Logs are key-value pairs, not just strings
2. Context preservation: Metadata follows logs through async operations
3. Performance: Lazy evaluation of log messages
4. Integration: Works seamlessly with FastAPI and async code
"""

import logging
import sys
from typing import Any

import structlog
from structlog.types import EventDict, Processor

from app.config import settings


def drop_color_message_key(_, __, event_dict: EventDict) -> EventDict:
    """
    Remove Uvicorn's color_message field from logs.
    
    Uvicorn duplicates messages with ANSI color codes, which we don't need
    in structured logs.
    """
    event_dict.pop("color_message", None)
    return event_dict


def add_app_context(_, __, event_dict: EventDict) -> EventDict:
    """
    Add application-level context to all logs.
    
    This processor automatically adds:
    - Environment (dev/staging/prod)
    - Application name
    - Service version (if available)
    
    Useful for:
    - Filtering logs by environment in centralized logging
    - Identifying service in microservices architecture
    """
    event_dict["environment"] = settings.ENVIRONMENT
    event_dict["app"] = settings.APP_NAME
    return event_dict


def setup_logging() -> None:
    """
    Configure structlog for the application.
    
    This function should be called once at application startup, before any
    logging occurs. It configures both structlog and the standard library
    logging to work together.
    
    Logging Flow:
    1. Structlog captures log with context
    2. Shared processors add metadata (timestamp, log level, etc.)
    3. ProcessorFormatter wraps for standard library compatibility
    4. Final renderer outputs JSON (prod) or console (dev)
    """
    
    # =========================================================================
    # CONFIGURE SHARED PROCESSORS
    # =========================================================================
    # These processors run on ALL log entries, regardless of source
    
    timestamper = structlog.processors.TimeStamper(fmt="iso")
    
    shared_processors: list[Processor] = [
        # Merge context variables (request ID, user ID, etc.)
        structlog.contextvars.merge_contextvars,
        
        # Add logger name (e.g., "app.agents.supervisor")
        structlog.stdlib.add_logger_name,
        
        # Add log level (DEBUG, INFO, WARNING, ERROR, CRITICAL)
        structlog.stdlib.add_log_level,
        
        # Format positional arguments in log messages
        structlog.stdlib.PositionalArgumentsFormatter(),
        
        # Add extra fields from logging.LogRecord
        structlog.stdlib.ExtraAdder(),
        
        # Remove Uvicorn's duplicate color message
        drop_color_message_key,
        
        # Add ISO 8601 timestamp
        timestamper,
        
        # Render stack traces for exceptions
        structlog.processors.StackInfoRenderer(),
        
        # Add application context
        add_app_context,
    ]
    
    # =========================================================================
    # CONFIGURE EXCEPTION FORMATTING
    # =========================================================================
    # For JSON logs, format exceptions as structured data
    # For console logs, pretty-print with colors
    
    if settings.LOG_FORMAT == "json":
        shared_processors.append(structlog.processors.format_exc_info)
    
    # =========================================================================
    # CONFIGURE STRUCTLOG
    # =========================================================================
    structlog.configure(
        processors=shared_processors + [
            # Final processor: wrap for standard library compatibility
            structlog.stdlib.ProcessorFormatter.wrap_for_formatter,
        ],
        # Use standard library LoggerFactory for compatibility
        logger_factory=structlog.stdlib.LoggerFactory(),
        # Cache logger instances for performance
        cache_logger_on_first_use=True,
    )
    
    # =========================================================================
    # CONFIGURE OUTPUT RENDERER
    # =========================================================================
    # JSON renderer for production (machine-readable)
    # Console renderer for development (human-readable with colors)
    
    log_renderer: structlog.types.Processor
    if settings.LOG_FORMAT == "json":
        log_renderer = structlog.processors.JSONRenderer()
    else:
        log_renderer = structlog.dev.ConsoleRenderer(
            colors=True,  # ANSI color codes
            exception_formatter=structlog.dev.plain_traceback,
        )
    
    # =========================================================================
    # CONFIGURE STANDARD LIBRARY LOGGING
    # =========================================================================
    # This ensures all logs (including from third-party libraries) use structlog
    
    formatter = structlog.stdlib.ProcessorFormatter(
        # These run ONLY on `logging` entries that do NOT originate within structlog
        foreign_pre_chain=shared_processors,
        
        # These run on ALL entries after the pre_chain
        processors=[
            # Remove internal structlog metadata
            structlog.stdlib.ProcessorFormatter.remove_processors_meta,
            
            # Final rendering (JSON or Console)
            log_renderer,
        ],
    )
    
    # =========================================================================
    # SETUP ROOT LOGGER
    # =========================================================================
    handler = logging.StreamHandler(sys.stdout)
    handler.setFormatter(formatter)
    
    root_logger = logging.getLogger()
    root_logger.addHandler(handler)
    root_logger.setLevel(settings.LOG_LEVEL.upper())
    
    # =========================================================================
    # CONFIGURE UVICORN LOGGERS
    # =========================================================================
    # Uvicorn has its own loggers that need special handling
    
    for logger_name in ["uvicorn", "uvicorn.error"]:
        # Clear existing handlers to avoid duplicate logs
        uvicorn_logger = logging.getLogger(logger_name)
        uvicorn_logger.handlers.clear()
        
        # Propagate to root logger (which uses our structlog formatter)
        uvicorn_logger.propagate = True
    
    # Disable uvicorn.access logger (we'll log requests in middleware)
    access_logger = logging.getLogger("uvicorn.access")
    access_logger.handlers.clear()
    access_logger.propagate = False
    
    # =========================================================================
    # LOG STARTUP MESSAGE
    # =========================================================================
    log = structlog.get_logger()
    log.info(
        "Logging configured",
        log_level=settings.LOG_LEVEL,
        log_format=settings.LOG_FORMAT,
        structured=settings.ENABLE_STRUCTURED_LOGGING,
    )


def get_logger(name: str = __name__) -> Any:
    """
    Get a structlog logger instance.
    
    Usage:
        from app.utils.logger import get_logger
        
        log = get_logger(__name__)
        log.info("Processing request", user_id=123, action="login")
    
    Args:
        name: Logger name (typically __name__ of the module)
    
    Returns:
        Structlog logger instance
    """
    return structlog.get_logger(name)


def bind_context(**kwargs: Any) -> None:
    """
    Bind context variables to the current async context.
    
    All subsequent logs in the same async context will include these variables.
    
    Usage:
        bind_context(request_id="abc-123", user_id=456)
        log.info("User logged in")  # Will include request_id and user_id
    
    Args:
        **kwargs: Key-value pairs to bind to context
    """
    structlog.contextvars.bind_contextvars(**kwargs)


def clear_context() -> None:
    """
    Clear all context variables from the current async context.
    
    Typically called at the beginning of each request to prevent context leakage.
    """
    structlog.contextvars.clear_contextvars()
```

***

### **File 4: `backend/app/main.py`**

```python
"""
FastAPI Application Entry Point

This is the main FastAPI application that:
1. Initializes logging
2. Configures middleware (CORS, request logging, error handling)
3. Mounts API routes
4. Provides health check endpoint
"""

import uuid
from contextlib import asynccontextmanager

from fastapi import FastAPI, Request, status
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse

from app.config import settings
from app.utils.logger import bind_context, clear_context, get_logger, setup_logging

# =========================================================================
# LOGGING SETUP (must be first)
# =========================================================================
setup_logging()
log = get_logger(__name__)


# =========================================================================
# LIFESPAN CONTEXT MANAGER
# =========================================================================
@asynccontextmanager
async def lifespan(app: FastAPI):
    """
    Lifespan events for FastAPI application.
    
    This context manager handles startup and shutdown events:
    - Startup: Initialize database connections, load models, etc.
    - Shutdown: Close connections, cleanup resources, etc.
    """
    # Startup
    log.info(
        "Application starting",
        environment=settings.ENVIRONMENT,
        debug=settings.DEBUG,
    )
    
    # TODO: Initialize database connection pool (Step 1.2)
    # TODO: Initialize Redis connection (Step 1.2)
    # TODO: Initialize vector store (Step 1.2)
    
    yield  # Application runs here
    
    # Shutdown
    log.info("Application shutting down")
    
    # TODO: Close database connections
    # TODO: Close Redis connection
    # TODO: Cleanup temporary files


# =========================================================================
# FASTAPI APP INITIALIZATION
# =========================================================================
app = FastAPI(
    title=settings.APP_NAME,
    description="Autonomous Multi-Agent Enterprise Intelligence System",
    version="0.1.0",
    docs_url="/docs" if settings.DEBUG else None,  # Disable docs in production
    redoc_url="/redoc" if settings.DEBUG else None,
    lifespan=lifespan,
)

# =========================================================================
# MIDDLEWARE CONFIGURATION
# =========================================================================

# CORS Middleware (allows frontend to communicate with backend)
app.add_middleware(
    CORSMiddleware,
    allow_origins=settings.CORS_ORIGINS_LIST,
    allow_credentials=True,
    allow_methods=["*"],  # Allow all HTTP methods
    allow_headers=["*"],  # Allow all headers
)


@app.middleware("http")
async def logging_middleware(request: Request, call_next):
    """
    Request logging middleware.
    
    This middleware:
    1. Generates unique request ID
    2. Binds request context (method, path, client IP)
    3. Logs request start and completion
    4. Measures request duration
    5. Clears context after request
    """
    # Clear any existing context (prevents leakage between requests)
    clear_context()
    
    # Generate unique request ID
    request_id = str(uuid.uuid4())
    
    # Bind context for this request
    bind_context(
        request_id=request_id,
        method=request.method,
        path=request.url.path,
        client_ip=request.client.host if request.client else "unknown",
    )
    
    # Log request start
    log.info("Request started")
    
    try:
        # Process request
        response = await call_next(request)
        
        # Log request completion
        log.info(
            "Request completed",
            status_code=response.status_code,
        )
        
        # Add request ID to response headers
        response.headers["X-Request-ID"] = request_id
        
        return response
    
    except Exception as exc:
        log.error(
            "Request failed",
            exc_info=exc,
            error=str(exc),
        )
        raise
    
    finally:
        # Clear context after request
        clear_context()


# =========================================================================
# HEALTH CHECK ENDPOINT
# =========================================================================
@app.get(
    "/api/health",
    tags=["Health"],
    summary="Health check endpoint",
    response_description="Service health status",
)
async def health_check():
    """
    Health check endpoint for monitoring and load balancers.
    
    Returns:
        dict: Service status and metadata
    """
    return {
        "status": "healthy",
        "environment": settings.ENVIRONMENT,
        "version": "0.1.0",
    }


@app.get(
    "/",
    tags=["Root"],
    summary="Root endpoint",
)
async def root():
    """
    Root endpoint.
    
    Returns:
        dict: Welcome message
    """
    return {
        "message": "Autonomous Multi-Agent Enterprise Intelligence System",
        "docs": "/docs",
        "health": "/api/health",
    }


# =========================================================================
# ERROR HANDLERS
# =========================================================================
@app.exception_handler(Exception)
async def global_exception_handler(request: Request, exc: Exception):
    """
    Global exception handler for unhandled errors.
    
    This catches all exceptions that aren't handled elsewhere and:
    1. Logs the error with full context
    2. Returns a consistent error response
    3. Prevents sensitive information leakage
    """
    log.error(
        "Unhandled exception",
        exc_info=exc,
        error_type=type(exc).__name__,
        error_message=str(exc),
    )
    
    return JSONResponse(
        status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
        content={
            "error": "Internal server error",
            "message": str(exc) if settings.DEBUG else "An unexpected error occurred",
        },
    )


# =========================================================================
# MOUNT ROUTERS (will be added in Step 1.5)
# =========================================================================
# TODO: app.include_router(agents_router, prefix="/api/agents", tags=["Agents"])
# TODO: app.include_router(conversations_router, prefix="/api/conversations")
# TODO: app.include_router(websocket_router, prefix="/ws")


if __name__ == "__main__":
    import uvicorn
    
    uvicorn.run(
        "app.main:app",
        host=settings.API_HOST,
        port=settings.API_PORT,
        reload=settings.DEBUG,
        log_level=settings.LOG_LEVEL.lower(),
    )
```

***

## **CHECKPOINT TEST: Verify Configuration & Logging**

### **Commands to execute from `backend/` directory:**

```bash
# Activate virtual environment
source .venv/bin/activate

# Test configuration loading
python -c "from app.config import settings; print(f'✓ Config loaded: {settings.APP_NAME}')"

# Test logger setup
python -c "from app.utils.logger import setup_logging, get_logger; setup_logging(); log = get_logger(); log.info('Test log'); print('✓ Logger working')"

# Start FastAPI server
uv run uvicorn app.main:app --reload --host 0.0.0.0 --port 8000

# In another terminal, test health endpoint
curl http://localhost:8000/api/health

# Expected output:
# {"status":"healthy","environment":"development","version":"0.1.0"}

# Test root endpoint
curl http://localhost:8000/

# Test with request ID tracking
curl -v http://localhost:8000/api/health | grep -i "x-request-id"

# View logs (should show structured JSON or colored console output)
# Logs should include: request_id, method, path, client_ip, timestamp
```

***

## **WHAT YOU'VE LEARNED**

1. ✅ **Pydantic Settings**: Type-safe configuration with automatic environment variable loading
2. ✅ **Computed Properties**: Dynamic values (DATABASE_URL, REDIS_URL, CORS_ORIGINS_LIST)
3. ✅ **Structlog**: Structured logging with context preservation across async operations
4. ✅ **FastAPI Lifespan**: Proper startup/shutdown handling
5. ✅ **Middleware**: Request logging, CORS, context binding
6. ✅ **Error Handling**: Global exception handler with structured error logging

***

**Reply "done" when your FastAPI server is running and all health checks pass, then we'll proceed to Step 1.2: Database Layer**.[2][3][4][5][1]

[1](https://fastapi.tiangolo.com/advanced/settings/)
[2](https://wazaari.dev/blog/fastapi-structlog-integration)
[3](https://pythonic.blog/2025/02/14/pydantic-series-settings-in-a-fastapi-app/)
[4](https://betterstack.com/community/guides/logging/structlog/)
[5](https://www.angelospanag.me/blog/structured-logging-using-structlog-and-fastapi)
[6](https://stackoverflow.com/questions/76674272/pydantic-basesettings-cant-find-env-when-running-commands-from-different-places)
[7](https://docs.pydantic.dev/latest/api/pydantic_settings/)
[8](https://github.com/abetlen/llama-cpp-python/issues/457)
[9](https://github.com/fastapi/fastapi/discussions/9913)
[10](https://gist.github.com/nymous/f138c7f06062b7c43c060bf03759c29e)
[11](https://dev.to/amal/pydantic-for-fastapi-2385)
[12](https://pypi.org/project/fastapi-structlog/)
[13](https://docs.pydantic.dev/latest/concepts/pydantic_settings/)
[14](https://www.dash0.com/guides/python-logging-with-structlog)
[15](https://last9.io/blog/fastapi-python/)
[16](https://ouassim.tech/notes/setting-up-structured-logging-in-fastapi-with-structlog/)